Spam Filter 0.1

Imports

In [1]:
import nltk
nltk.data.path.append('/home/toutou/nltk_data/')
import os
import codecs
import re
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tkinter import *
import pandas as pd
import numpy as np
import glob, os
data = pd.read_csv("test.csv")
root = Tk()

In [2]:
data

,type,text
0,s,reformolarprezado senhor a tomamos a liberdade...
1,s,very important messagefrom johnson werner bill...
2,s,principle vlcodd n vlaagrra claal s somma alli...
3,s,vicodin and other inexpnsive medlcations no pr...
4,s,hidear sir madam i am engr munachim eze chairm...
5,s,re time to reorder cia lishi again we now have...
6,s,is the time right for this smallcap executive ...
7,s,mr cole cuts your monthly paymentshello i sent...
8,s,stox to watchinvestor alert newsletter july is...
9,s,win a dollar wal mart gift card unable to see ...


Variabls

In [4]:
#Global
language = IntVar()
data = pd.read_csv("test.csv")

#Bayesian Model
spamWordCount = {}
spamWordTotal = 0
hamWordCount = {}
hamWordTotal = 0
pSpam = 0.5
pHam = 0.5

#UI
emailLabel = Label(root, text="Sender :")
emailVar = StringVar()
emailEntry = Entry(root, textvariable=emailVar)
subjectLabel = Label(root, text="Subject :")
subjectVar = StringVar()
subjectEntry = Entry(root, textvariable=subjectVar)
contentLabel = Label(root, text="Content :")
contentText = Text(root)
contactButton = Button(root, text="Contact")
whitelistVar = StringVar()
contactCheckbutton = Checkbutton(root, text="Activate WhiteList", variable=whitelistVar, onvalue="on", offvalue="off")
frRadiobutton = Radiobutton(root, text="Français", variable=language, value=1)
engRadiobutton = Radiobutton(root, text="English",variable=language, value=2)
language.set(2)
chargerVar = StringVar()
csvVar = StringVar()
os.chdir(".")
TOPTIONS = glob.glob("*.txt")
COPTIONS = glob.glob("*.csv")
chargerOptionMenu = OptionMenu(root, chargerVar, *TOPTIONS)
csvOptionMenu = OptionMenu(root, csvVar, *COPTIONS)
resultVar = StringVar()
resultLabel = Label(root, textvariable=resultVar)


#Text Processing
stemmer = PorterStemmer()
french_stemmer = SnowballStemmer('french')



Functions

In [6]:
#Bayesian Train function
def Train(row) :
    global spamWordCount
    global spamWordTotal
    global hamWordCount
    global hamWordTotal
    email = row['text']
    if row['type']=='s' :
        for word in email :
            spamWordCount[word] = spamWordCount.get(word, 0) + 1 
            spamWordTotal += 1
    if row['type']=='h' :
        for word in email :
            hamWordCount[word] = hamWordCount.get(word, 0) + 1
            hamWordTotal += 1

In [7]:
#UI Button
def generateModel() :
    global spamWordCount
    global spamWordTotal
    global hamWordCount
    global hamWordTotal
    global pSpam
    global pHam
    spamWordCount.clear()
    hamWordCount.clear()
    spamWordTotal = 0
    hamWordTotal = 0
    data.apply(Train, axis=1)
    pSpam = spamWordTotal/(spamWordTotal+hamWordTotal)
    pHam = 1-pSpam

In [8]:
#Processing
def preprocess(raw) :
    lang = language.get()
    if lang == 2 :
        s = str(raw)
        wordlist = word_tokenize(s)
        returnlist = []
        for word in wordlist :
            if (word not in stopwords.words("english")) :
                if (word[0] in (('1','2','3','4','5','6','7','8','9','0','&','(','-','_','ç','é','à',')','=','+','°','~','#'))) :
                    returnlist.append('#')
                else :
                    returnlist.append(stemmer.stem(word))
        return ' '.join(returnlist)
    if lang == 1 :
        s = str(raw)
        wordlist = word_tokenize(raw,"french")
        returnlist = []
        for word in wordlist :
            if (word not in stopwords.words("french")) :
                if (word[0] in (('1','2','3','4','5','6','7','8','9','0','&','(','-','_','ç','é','à',')','=','+','°','~','#'))) :
                    returnlist.append('#')
                else :
                    returnlist.append(french_stemmer.stem(word))
        return ' '.join(returnlist)

In [10]:
#Import CSV
def importCSV() :
    global data
    name = csvVar.get()
    dataX=pd.read_csv(name)
    data = dataX.dropna()
    generateModel()

In [11]:
#Load Contacts
def loadContacts():
    global contacts
    contacts = pd.read_csv("contacts.csv")
    contacts = contacts['E-mail Address']

In [12]:
#Clean Dataset
def cleanData():
    global data
    global COPTIONS
    dat = data.dropna()
    da = dat['text'].apply(preprocess)
    pd.DataFrame({'type': data.type, 'text': da}).to_csv("generated.csv")
    COPTIONS = glob.glob("*.csv")

In [13]:
#Load Mail
def loadMail() :
    choice = chargerVar.get()
    text_f = []
    if(choice in os.listdir('.')) :
        with codecs.open(choice, "r",encoding='utf-8', errors='ignore') as f:
            lignes  = f.readlines()
            lignes[1].split('<')
            for elm in lignes[1].split('<'):
                for elm2 in elm.split('>'):
                    for c in elm2:
                        if c=="@":
                            email=elm2
            subject=lignes[3].split(':')[1]
            for line in lignes[5:]:
                text_f.extend(line)
            emailVar.set(email)
            subjectVar.set(subject)
            contentText.insert(END, ''.join(text_f))

In [14]:
#Classifying
def classify() :
    if whitelistVar.get() == "on" :
        if emailVar.get() not in contacts :
            resultVar.set("WhiteList Filtered")
        else :
            resultVat.set("WhiteList Contact")
    else :
        spamScore = 1
        hamScore = 1
        textInput = contentText.get("1.0",'end-1c')
        textReady = preprocess(textInput)
        for word in textInput.split() :
            spamScore *= spamWordCount.get(word, spamWordTotal)/spamWordTotal
            hamScore *= hamWordCount.get(word, hamWordTotal)/hamWordTotal
        if pSpam * spamScore > pHam * hamScore :
            resultVar.set("Result = SPAM")
        else :
            resultVar.set("Result = HAM")

Display UI

In [16]:
#Buttons
csvButton = Button(root, command=importCSV, text="Use Dataset")
chargerButton = Button(root, text="Load", command=loadMail)
cleanButton = Button(root, text="Clean Dataset", command=cleanData)
classifierButton = Button(root, text="Classify", command=classify)
whitelistVar.set("on")


#Layout
language.set(2)
contactButton.grid()
contactCheckbutton.grid(sticky=W, row=0, column=1)
frRadiobutton.grid(row=0, column=2)
engRadiobutton.grid(row=0, column=3)
csvOptionMenu.grid(row=0, column=4)
csvButton.grid(row=0, column=5)
emailLabel.grid(sticky=E)
subjectLabel.grid(sticky=E)
contentLabel.grid(sticky=NE)
emailEntry.grid(sticky=W, row=1, columnspan=2, column=1)
resultLabel.grid(row=1, column=3)
cleanButton.grid(row=1, column=5)
subjectEntry.grid(sticky=W, row=2, columnspan = 2,column=1)
contentText.grid(row=3, column=1, columnspan=5)
chargerButton.grid()
chargerOptionMenu.grid(row=4, column=1, rowspan=2)
classifierButton.grid(row=4, column=5)

In [ ]:
root.mainloop()